In [10]:
import numpy as np
import cv2
import os

X_train = []
data_path = 'C:/Users/Administrator/Desktop/lstm_result0605/Dataset1_sample'
#data_path = 'C:/Users/Administrator/Desktop/Dataset_noslice_sample'
#data_path = 'C:/Users/Administrator/Desktop/Dataset1_train'
#data_path = 'C:/Users/Administrator/Desktop/Dataset1_train_cut'
#data_path = 'C:/Users/Administrator/Desktop/Dataset1_cut_resize'
folder_list = os.listdir(data_path)
for folder in folder_list:
    folder_path = os.path.join(data_path, folder)
    file_list = os.listdir(folder_path)
    for file in file_list:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        pic = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        pic_list = pic.tolist()
        X_train.append(pic_list)
X_train_np = np.array(X_train)

In [11]:
X_train_np.shape

(7296, 92, 136)

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a path of crack."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [13]:
latent_dim = 100

encoder_inputs = keras.Input(shape=(92, 136, 1))
#encoder_inputs = keras.Input(shape=(56, 100, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(200, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 92, 136, 1)] 0                                            
______________________________________________________________________________________

In [14]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(23*34*64, activation="relu")(latent_inputs)
x = layers.Reshape((23, 34, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 50048)             5054848   
_________________________________________________________________
reshape_1 (Reshape)          (None, 23, 34, 64)        0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 46, 68, 64)        36928     
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 92, 136, 32)       18464     
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 92, 136, 1)        289       
Total params: 5,110,529
Trainable params: 5,110,529
Non-trainable params: 0
_________________________________________________

In [15]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [16]:
X_train_final = np.expand_dims(X_train_np, -1).astype("float32") / 255

In [17]:
X_train_final.shape

(7296, 92, 136, 1)

In [18]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(X_train_final, epochs=300, batch_size=128)

Epoch 1/300
57/57 [==============================] - 65s 1s/step - loss: 3720.2108 - reconstruction_loss: 1333.0515 - kl_loss: 121.8585
Epoch 2/300
57/57 [==============================] - 66s 1s/step - loss: 161.4443 - reconstruction_loss: 147.1633 - kl_loss: 10.8369
Epoch 3/300
57/57 [==============================] - 66s 1s/step - loss: 150.7994 - reconstruction_loss: 140.7818 - kl_loss: 8.9400
Epoch 4/300
57/57 [==============================] - 67s 1s/step - loss: 146.9154 - reconstruction_loss: 137.7200 - kl_loss: 8.7695
Epoch 5/300
57/57 [==============================] - 67s 1s/step - loss: 144.2584 - reconstruction_loss: 134.7388 - kl_loss: 8.5791
Epoch 6/300
57/57 [==============================] - 67s 1s/step - loss: 141.6928 - reconstruction_loss: 132.8765 - kl_loss: 8.2379
Epoch 7/300
57/57 [==============================] - 68s 1s/step - loss: 137.9095 - reconstruction_loss: 132.0652 - kl_loss: 6.6893
Epoch 8/300
57/57 [==============================] - 68s 1s/step - loss

57/57 [==============================] - 69s 1s/step - loss: 85.5397 - reconstruction_loss: 77.7257 - kl_loss: 8.1632
Epoch 125/300
57/57 [==============================] - 69s 1s/step - loss: 86.0680 - reconstruction_loss: 77.4682 - kl_loss: 8.1143
Epoch 126/300
57/57 [==============================] - 68s 1s/step - loss: 84.6904 - reconstruction_loss: 77.4266 - kl_loss: 8.1172
Epoch 127/300
57/57 [==============================] - 69s 1s/step - loss: 85.9910 - reconstruction_loss: 77.4267 - kl_loss: 8.1639
Epoch 128/300
57/57 [==============================] - 69s 1s/step - loss: 85.0819 - reconstruction_loss: 77.4663 - kl_loss: 8.1063
Epoch 129/300
57/57 [==============================] - 68s 1s/step - loss: 85.6013 - reconstruction_loss: 77.2755 - kl_loss: 8.2133
Epoch 130/300
57/57 [==============================] - 68s 1s/step - loss: 86.1920 - reconstruction_loss: 77.2751 - kl_loss: 8.1414
Epoch 131/300
57/57 [==============================] - 68s 1s/step - loss: 85.1689 - recon

57/57 [==============================] - 69s 1s/step - loss: 82.7037 - reconstruction_loss: 74.0943 - kl_loss: 8.4614
Epoch 249/300
57/57 [==============================] - 69s 1s/step - loss: 81.7565 - reconstruction_loss: 74.0746 - kl_loss: 8.3768
Epoch 250/300
57/57 [==============================] - 83s 1s/step - loss: 83.2030 - reconstruction_loss: 74.0894 - kl_loss: 8.4803
Epoch 251/300
57/57 [==============================] - 113s 2s/step - loss: 82.8307 - reconstruction_loss: 74.0798 - kl_loss: 8.4966
Epoch 252/300
57/57 [==============================] - 115s 2s/step - loss: 82.4125 - reconstruction_loss: 74.0706 - kl_loss: 8.4764
Epoch 253/300
57/57 [==============================] - 117s 2s/step - loss: 82.3925 - reconstruction_loss: 74.0309 - kl_loss: 8.4793
Epoch 254/300
57/57 [==============================] - 117s 2s/step - loss: 82.7576 - reconstruction_loss: 74.0617 - kl_loss: 8.4651
Epoch 255/300
57/57 [==============================] - 117s 2s/step - loss: 82.6929 - 

In [10]:
X_test = []
#data_path = 'C:/Users/Administrator/Desktop/Dataset1_test'
#data_path = 'C:/Users/Administrator/Desktop/Dataset1_train'
data_path = 'C:/Users/Administrator/Desktop/Dataset1_sample'
#data_path = 'C:/Users/Administrator/Desktop/Dataset_noslice_sample'
folder_list1 = os.listdir(data_path)
folder_list = sorted(folder_list1, key=lambda x: int(x[6:]))
for folder in folder_list:
    print(folder)
    folder_path = os.path.join(data_path, folder)
    file_list1 = os.listdir(folder_path)
    file_list = sorted(file_list1, key=lambda info: (int(info[0:-4]), info[-4:]))
    for file in file_list:
        print(file)
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        pic = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        pic_list = pic.tolist()
        X_test.append(pic_list)
X_test_np = np.array(X_test)
X_test_final = np.expand_dims(X_test_np, -1).astype("float32") / 255
#z_mean, _, _ = vae.encoder.predict(X_test_final)
#x_decoded = vae.decoder.predict(z_mean)
_, _, z_mean = vae.encoder.predict(X_test_final)
x_decoded = vae.decoder.predict(z_mean)
print(x_decoded.shape)
x_decoded_reshape = x_decoded.reshape(152, 92, 136)*255
#x_decoded_reshape = x_decoded.reshape(152, 56, 100)*255
print(x_decoded_reshape.shape)

reconstr_path = 'C:/Users/Administrator/Desktop/reconstruction/'
os.mkdir(reconstr_path)
for count in range(7296):
#for count in range(152):
    tmp = x_decoded_reshape[count,:,:]
    count_int = count//8
    #count_int_2 = count_int + 1
    count_residue = count%8
    count_residue_2 = count_residue + 1
    count_residue_3 = count_residue_2 * 5
    sample_name = folder_list[count_int]
    #sample_name = 'Sample'+ str(count_int_2)
    recon_file_path = os.path.join(reconstr_path, sample_name)
    file_name = str(count_residue_3)+'.png'
    recon_file_path2 = os.path.join(recon_file_path, file_name)
    if count_residue == 0:
        os.mkdir(recon_file_path)
        print(sample_name, 'is reconstrcting')
    cv2.imwrite(recon_file_path2,tmp)

#X_test= []
#test_file_path = 'C:/Users/Administrator/Desktop/Dataset1_test/Sample388/79.png'
#img = cv2.imread(test_file_path)
#pic = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
#pic_list = pic.tolist()
#X_test.append(pic_list)
#X_test_np = np.array(X_test)
#X_test_final = np.expand_dims(X_test_np, -1).astype("float32") / 255

Sample1
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample2
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample4
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample7
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample8
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample10
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample11
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample12
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample13
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample14
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample15
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample16
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample20
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample21
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample22
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample23
5.png
10.png
15.png
20.png
25.png
30

In [11]:
z_mean.shape

(152, 100)

In [12]:
z_display = z_mean[:,0:2]

In [13]:
z_display.shape

(152, 2)

In [14]:
np.savetxt('C:/Users/Administrator/Desktop/z_distribution.txt',z_display)

In [16]:
#import matplotlib.pyplot as plt
#def plot_label_clusters(vae, data):
    # display a 2D plot of the digit classes in the latent space
    #z_mean, _, _ = vae.encoder.predict(data)
    #plt.figure(figsize=(12, 10))
   # plt.scatter(z_mean[:, 0], z_mean[:, 1])
    #plt.xlabel("z[0]")
    #plt.ylabel("z[1]")
    #plt.show()

    



#plot_label_clusters(vae, X_train_final)

In [15]:
def reduce(path):
    img = cv2.imread(path)
    pic = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    pic_list = pic.tolist()
    return pic_list

train_X_pre = []
train_Y_pre = []
train_X = []
train_Y = []
#c_dir_path = 'C:/Users/Administrator/Desktop/Dataset1_train_vae'
c_dir_path = 'C:/Users/Administrator/Desktop/Dataset_noslice_train_vae'
#c_dir_path = 'C:/Users/Administrator/Desktop/Dataset1_sample_repeat'
p_c_dir_list = os.listdir(c_dir_path)
for p_c_dir in p_c_dir_list:
    dir_path = os.path.join(c_dir_path, p_c_dir)
    filelist1 = os.listdir(dir_path)
    filelist = sorted(filelist1, key=lambda info: (int(info[0:-4]), info[-4:]))
    print()
    #for i in range(8):
    for i in range(6):
        train_X_pre.append(reduce(os.path.join(dir_path, filelist[i])))
        train_X_pre.append(reduce(os.path.join(dir_path, filelist[i+1])))
        train_Y_pre.append(reduce(os.path.join(dir_path, filelist[i+2])))
        train_Y_pre.append(reduce(os.path.join(dir_path, filelist[i+3])))
        train_Y_pre.append(reduce(os.path.join(dir_path, filelist[i+4])))
        train_Y_pre.append(reduce(os.path.join(dir_path, filelist[i+5])))
        train_Y_pre.append(reduce(os.path.join(dir_path, filelist[i+6])))
        train_Y_pre.append(reduce(os.path.join(dir_path, filelist[i+7])))

In [16]:
train_X_pre_np = np.array(train_X_pre)
train_Y_pre_np = np.array(train_Y_pre)
train_X_unencode = np.expand_dims(train_X_pre_np, -1).astype("float32") / 255
train_Y_unencode = np.expand_dims(train_Y_pre_np, -1).astype("float32") / 255

In [17]:
train_X_np, _, _ = vae.encoder.predict(train_X_unencode)
train_Y_np, _, _ = vae.encoder.predict(train_Y_unencode)

In [18]:
train_X_np.shape

(180, 100)

In [19]:
train_Y_np.shape

(540, 100)

In [21]:
train_X_np = train_X_np.reshape((90, 2, 100))
train_Y_np = train_Y_np.reshape((90, 6, 100))

In [22]:
#np.save('C:/Users/Administrator/Desktop/train_X_np.npy', train_X_np)
#np.save('C:/Users/Administrator/Desktop/train_Y_np.npy', train_Y_np)
np.save('C:/Users/Administrator/Desktop/train_X_np_noslice.npy', train_X_np)
np.save('C:/Users/Administrator/Desktop/train_Y_np_noslice.npy', train_Y_np)

In [37]:
test_X_pre = []
test_Y_pre = []
test_X = []
test_Y = []
#c_dir_path = 'C:/Users/Administrator/Desktop/Dataset_noslice_test_vae'
c_dir_path = 'C:/Users/Administrator/Desktop/Dataset1_test_vae'
p_c_dir_list1 = os.listdir(c_dir_path)
p_c_dir_list = sorted(p_c_dir_list1, key=lambda x: int(x[6:]))
for p_c_dir in p_c_dir_list:
    dir_path = os.path.join(c_dir_path, p_c_dir)
    filelist1 = os.listdir(dir_path)
    filelist = sorted(filelist1, key=lambda info: (int(info[0:-4]), info[-4:]))
    for i in range(6):
        test_X_pre.append(reduce(os.path.join(dir_path, filelist[i])))
        test_X_pre.append(reduce(os.path.join(dir_path, filelist[i+1])))
        test_Y_pre.append(reduce(os.path.join(dir_path, filelist[i+2])))
        test_Y_pre.append(reduce(os.path.join(dir_path, filelist[i+3])))
        test_Y_pre.append(reduce(os.path.join(dir_path, filelist[i+4])))
        test_Y_pre.append(reduce(os.path.join(dir_path, filelist[i+5])))
        test_Y_pre.append(reduce(os.path.join(dir_path, filelist[i+6])))
        test_Y_pre.append(reduce(os.path.join(dir_path, filelist[i+7])))

In [38]:
test_X_pre_np = np.array(test_X_pre)
test_Y_pre_np = np.array(test_Y_pre)
test_X_unencode = np.expand_dims(test_X_pre_np, -1).astype("float32") / 255
test_Y_unencode = np.expand_dims(test_Y_pre_np, -1).astype("float32") / 255

In [39]:
test_X_np, _, _ = vae.encoder.predict(test_X_unencode)
test_Y_np, _, _ = vae.encoder.predict(test_Y_unencode)

In [40]:
test_X_np.shape

(1344, 100)

In [41]:
test_Y_np.shape

(4032, 100)

In [42]:
#test_X_np = test_X_np.reshape((24, 2, 100))
#test_Y_np = test_Y_np.reshape((24, 6, 100))
test_X_np = test_X_np.reshape((672, 2, 100))
test_Y_np = test_Y_np.reshape((672, 6, 100))

In [43]:
#np.save('C:/Users/Administrator/Desktop/test_X_np.npy', test_X_np)
#np.save('C:/Users/Administrator/Desktop/test_Y_np.npy', test_Y_np)
#np.save('C:/Users/Administrator/Desktop/test_X_np_noslice.npy', test_X_np)
#np.save('C:/Users/Administrator/Desktop/test_Y_np_noslice.npy', test_Y_np)
np.save('C:/Users/Administrator/Desktop/test_X_np_noslice_rare.npy', test_X_np)
np.save('C:/Users/Administrator/Desktop/test_Y_np_noslice_rare.npy', test_Y_np)

In [19]:
#vae.decoder.save('C:/Users/Administrator/Desktop/decoder.h5')
vae.decoder.save('C:/Users/Administrator/Desktop/decoder_300_128.h5')

In [20]:
#vae.encoder.save('C:/Users/Administrator/Desktop/encoder.h5')
vae.encoder.save('C:/Users/Administrator/Desktop/encoder_300_128.h5')

In [32]:
train_X_life_pre = []
#data_path = 'C:/Users/Administrator/Desktop/Dataset1_train_life'
data_path = 'C:/Users/Administrator/Desktop/Dataset_noslice_train_life'
folder_list1 = os.listdir(data_path)
folder_list = sorted(folder_list1, key=lambda x: int(x[6:]))
for folder in folder_list:
    print(folder)
    folder_path = os.path.join(data_path, folder)
    file_list1 = os.listdir(folder_path)
    file_list = sorted(file_list1, key=lambda info: (int(info[0:-4]), info[-4:]))
    for file in file_list:
        print(file)
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        pic = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        pic_list = pic.tolist()
        train_X_life_pre.append(pic_list)
train_X_life_pre_np = np.array(train_X_life_pre)
train_X_life_pre_final = np.expand_dims(train_X_life_pre_np, -1).astype("float32") / 255
z_mean_train, _, _ = vae.encoder.predict(train_X_life_pre_final)
print(z_mean_train.shape)
#x_decoded = vae.decoder.predict(z_mean)
#print(x_decoded.shape)
#x_decoded_reshape = x_decoded.reshape(8000, 92, 136)*255
#print(x_decoded_reshape.shape)

Sample1
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample2
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample4
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample7
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample8
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample10
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample11
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample12
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample13
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample14
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample15
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample16
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample20
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample21
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample22
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
(120, 100)


In [34]:
#np.save('C:/Users/Administrator/Desktop/train_X_life_np.npy', z_mean_train)
np.save('C:/Users/Administrator/Desktop/train_X_life_np_noslice.npy', z_mean_train)

In [35]:
test_X_life_pre = []
#data_path = 'C:/Users/Administrator/Desktop/Dataset1_test_life'
data_path = 'C:/Users/Administrator/Desktop/Dataset_noslice_test_life'
folder_list1 = os.listdir(data_path)
folder_list = sorted(folder_list1, key=lambda x: int(x[6:]))
for folder in folder_list:
    print(folder)
    folder_path = os.path.join(data_path, folder)
    file_list1 = os.listdir(folder_path)
    file_list = sorted(file_list1, key=lambda info: (int(info[0:-4]), info[-4:]))
    for file in file_list:
        print(file)
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        pic = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        pic_list = pic.tolist()
        test_X_life_pre.append(pic_list)
test_X_life_pre_np = np.array(test_X_life_pre)
test_X_life_pre_final = np.expand_dims(test_X_life_pre_np, -1).astype("float32") / 255
z_mean_test, _, _ = vae.encoder.predict(test_X_life_pre_final)
print(z_mean_test.shape)
#x_decoded = vae.decoder.predict(z_mean)
#print(x_decoded.shape)
#x_decoded_reshape = x_decoded.reshape(8000, 92, 136)*255
#print(x_decoded_reshape.shape)

Sample23
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample25
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample29
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
Sample30
5.png
10.png
15.png
20.png
25.png
30.png
35.png
40.png
(32, 100)


In [36]:
#np.save('C:/Users/Administrator/Desktop/test_X_life_np.npy', z_mean_test)
np.save('C:/Users/Administrator/Desktop/test_X_life_np_noslice.npy', z_mean_test)